# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_key

from config import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
weather_stat = pd.read_csv("cities.csv")
weather_stat


,Unnamed: 0,city,latitude,longitude,temperature,humidity,cloudiness,windspeed,date
0,0,Mar del Plata,-38.00,-57.56,55.24,58,90,5.82,1601506578
1,1,Punta Arenas,-53.15,-70.92,37.20,47,20,25.28,1601506833
2,2,Kirkuk,35.47,44.39,74.50,25,0,4.94,1601507020
3,3,Puerto Ayora,-0.74,-90.35,66.99,91,86,8.01,1601506893
4,4,Busselton,-33.65,115.33,59.45,67,100,13.80,1601506819
...,...,...,...,...,...,...,...,...,...
586,586,Boa Vista,2.82,-60.67,89.60,49,20,3.36,1601507253
587,587,Huéscar,37.81,-2.54,63.97,49,68,2.77,1601507253
588,588,Nagato,34.38,131.20,66.33,88,20,3.36,1601507254
589,589,Hurghada,27.26,33.81,89.60,21,1,5.82,1601506661


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_stat[["latitude", "longitude"]]

# Store Humidity in humidity
humidity = weather_stat["humidity"]

In [68]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### max_intensity


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
fitted_weather_df = weather_stat.loc[(weather_stat["windspeed"] >= 10) & 
                                    (weather_stat["cloudiness"] == 0) & \
                                   (weather_stat["temperature"] >= 60) & 
                                    (weather_stat["temperature"] <= 80)].dropna()

fitted_weather_df

,Unnamed: 0,city,latitude,longitude,temperature,humidity,cloudiness,windspeed,date
79,79,Yulara,-25.24,130.99,68.00,32,0,11.41,1601507048
88,88,Nouadhibou,20.93,-17.03,75.60,74,0,20.18,1601507052
139,139,Alice Springs,-23.70,133.88,72.73,26,0,10.29,1601507052
192,192,Ballitoville,-29.54,31.21,62.60,77,0,12.75,1601507093
263,263,Currais Novos,-6.26,-36.52,72.09,72,0,18.03,1601507121
435,435,Bolivar,37.61,-93.41,75.04,35,0,10.42,1601507192
564,564,Tsabong,-26.02,22.40,63.28,22,0,12.93,1601507242


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
# Create a hotel_df
hotel_df = fitted_weather_df.loc[:,["city","latitude", "longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["hotel name"] = ""

# Display the result
hotel_df

,city,latitude,longitude,hotel name
79,Yulara,-25.24,130.99,
88,Nouadhibou,20.93,-17.03,
139,Alice Springs,-23.70,133.88,
192,Ballitoville,-29.54,31.21,
263,Currais Novos,-6.26,-36.52,
435,Bolivar,37.61,-93.41,
564,Tsabong,-26.02,22.40,


In [74]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [77]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["latitude"]
    lng = row["longitude"]
    city_name = row["city"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "hotel name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 79: Yulara.
Closest hotel in Yulara is Sails in the Desert.
------------
Retrieving Results for Index 88: Nouadhibou.
Closest hotel in Nouadhibou is Free Zone Hotel Nouadhibou.
------------
Retrieving Results for Index 139: Alice Springs.
Closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
------------
Retrieving Results for Index 192: Ballitoville.
Closest hotel in Ballitoville is Zimbali Lodge.
------------
Retrieving Results for Index 263: Currais Novos.
Closest hotel in Currais Novos is Tungstenio Hotel.
------------
Retrieving Results for Index 435: Bolivar.
Closest hotel in Bolivar is Best Western Plus Bolivar Hotel & Suites.
------------
Retrieving Results for Index 564: Tsabong.
Closest hotel in Tsabong is Dikukama Hotel.
------------
-------End of Search-------


In [78]:
# Display the hotel dataframe
hotel_df

,city,latitude,longitude,hotel name
79,Yulara,-25.24,130.99,Sails in the Desert
88,Nouadhibou,20.93,-17.03,Free Zone Hotel Nouadhibou
139,Alice Springs,-23.70,133.88,DoubleTree by Hilton Hotel Alice Springs
192,Ballitoville,-29.54,31.21,Zimbali Lodge
263,Currais Novos,-6.26,-36.52,Tungstenio Hotel
435,Bolivar,37.61,-93.41,Best Western Plus Bolivar Hotel & Suites
564,Tsabong,-26.02,22.40,Dikukama Hotel


In [91]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>name</dt><dd>{hotel name}</dd>
<dt>city</dt><dd>{city}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [92]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))